In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Import our custom functions
from src.generators import (
    homogeneous_poisson,
    homogeneous_poisson_with_refractoriness,
    homogeneous_poisson_batch,
    refractory_poisson_batch,
    interspike_intervals
)

from src.analysis import (
    coefficient_of_variation,
    fano_factor,
    fano_factor_across_timescales,
    isi_histogram,
    firing_rate_estimate
)

# Set style for nicer plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

print("✓ All imports successful!")

✓ All imports successful!


print("\n" + "="*70)
print("PART 2: REFRACTORY PERIOD - BIOLOGICAL REALISM")
print("="*70)

print("""
What is a refractory period?
---------------------------
After a neuron fires an action potential, the ion channels need time to reset.
During this time, the neuron CANNOT fire again, no matter how strong the input.

This is called the ABSOLUTE REFRACTORY PERIOD.

Typical duration: 2-5 milliseconds

Why it matters:
- Prevents extremely rapid firing
- Makes spike timing more regular (less random)
- Changes the distribution of interspike intervals
- Creates structure in spike trains

In this notebook, we'll:
1. Generate Poisson spikes with a refractory period constraint
2. Compare to pure Poisson (no constraints)
3. See how statistics change
4. Visualize the differences
""")
